In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **굴절률 데이터 가져오기 (TMM으로 데이터 생성시 필요)**

In [ ]:
import numpy as np
import pandas as pd
import os

# 파장 정의
WAVELENGTHS = np.arange(380, 781, 1)
NUM_WAVELENGTHS = len(WAVELENGTHS) #401

# GitHub raw 주소 (파일들이 위치한 경로)
base_url = "https://raw.githubusercontent.com/KIMEUIJOON-KNU/Bi-directional/main/"

# 사용할 재료 목록
material_files = [
    ('SiO2', 'SiO2_380.csv'),
    ('WO3',  'WO3_380.csv'),
    ('Ag',   'Ag_380.csv')
]

material_data = {}
material_names = [info[0] for info in material_files]
print("재료 리스트:", material_names)

# 재료 인덱싱 매핑
material_names = sorted(set(material_names))
material_to_index = {name: i for i, name in enumerate(material_names)}
index_to_material = {i: name for name, i in material_to_index.items()}
print("재료 인덱스 매핑:", index_to_material)

# --- 재료 파일 불러오기 ---
for material_name, filename in material_files:
    file_url = base_url + filename
    print(f"처리 중: {filename} (materials: {material_name}) → {file_url}")
    try:
        data = pd.read_csv(file_url, header=None)
    except Exception as e:
        print(f"  🚫 오류 발생: {e}")
        continue

    if data.empty:
        print("  ⚠️ 경고: 파일이 비어있습니다.")
        continue

    wavelengths = data.iloc[:, 0].values  # 파장
    n = data.iloc[:, 1].values           # 실수부
    k = data.iloc[:, 2].values           # 허수부

    n_complex = n + 1j * k               # 복소 굴절률

    material_data[material_name] = (wavelengths, n_complex)
    print(f"로드 완료: {len(wavelengths)} points")

# --- 공기 굴절률 추가 ---
material_data['Air'] = np.ones(NUM_WAVELENGTHS, dtype=np.complex128)
print("Air 굴절률 추가 완료")


재료 리스트: ['SiO2', 'WO3', 'Ag']
재료 인덱스 매핑: {0: 'Ag', 1: 'SiO2', 2: 'WO3'}
처리 중: SiO2_380.csv (materials: SiO2) → https://raw.githubusercontent.com/KIMEUIJOON-KNU/Bi-directional/main/SiO2_380.csv
로드 완료: 401 points
처리 중: WO3_380.csv (materials: WO3) → https://raw.githubusercontent.com/KIMEUIJOON-KNU/Bi-directional/main/WO3_380.csv
로드 완료: 401 points
처리 중: Ag_380.csv (materials: Ag) → https://raw.githubusercontent.com/KIMEUIJOON-KNU/Bi-directional/main/Ag_380.csv
로드 완료: 401 points
Air 굴절률 추가 완료


In [ ]:
# ────────────────────────────────────────────────────────────────────────────
# (1) 파장 정의 (380~780 nm, 1 nm 간격 → 총 401개 파장 지점)
# ────────────────────────────────────────────────────────────────────────────
λ_tensor_global = torch.tensor(WAVELENGTHS, dtype=torch.float64).to(device) * 1e-9  # [m 단위로 변환된 파장 텐서]

# ────────────────────────────────────────────────────────────────────────────
# (2) 층별 재료 정의 (MIM 구조 예시: 금속-절연체-금속)
# ────────────────────────────────────────────────────────────────────────────
material_sequence = ['Ag', 'WO3', 'Ag']  # 구조에 사용될 재료 순서 (입사면에서부터 아래 방향)

# material_data 에서 각 재료의 복소 굴절률 (n + ik) 값 추출 (shape: [401])
n_list_np_arrays = [material_data[mat][1] for mat in material_sequence]  # 길이 3 리스트 (각각 [401] 크기)

# numpy → torch 변환 후, complex64 형식으로 묶어서 GPU로 이동 (shape: [3, 401])
n_list_torch = torch.stack([
    torch.from_numpy(arr).to(torch.complex64) for arr in n_list_np_arrays
], dim=0).to(device)

# ────────────────────────────────────────────────────────────────────────────
# (3) 입사 매질 (n_i)와 출사 매질 (n_s) 설정
# ────────────────────────────────────────────────────────────────────────────
n_i_np = material_data['SiO2'][1]   # 입사 매질: SiO2의 복소 굴절률 (n + ik)
n_s_np = material_data['Air']       # 출사 매질: Air (n=1, k=0)

n_i_torch = torch.from_numpy(n_i_np).to(torch.complex64).to(device)  # torch 변환 후 GPU 이동
n_s_torch = torch.from_numpy(n_s_np).to(torch.complex64).to(device)

# ────────────────────────────────────────────────────────────────────────────
# (4) 두께 그리드 정의 (nm 단위, 총 조합 수 = 8 * 187 * 8 = 11968개)
# ────────────────────────────────────────────────────────────────────────────
d1_list_nm = np.arange(5, 41, 5)      # 금속층 (d1): 5 ~ 40 nm, 간격 5 nm → 총 8개
d2_list_nm = np.arange(150, 801, 5)    # 절연층 (d2): 20 ~ 950 nm, 간격 5 nm → 총 187개
d3_list_nm = np.arange(5, 41, 5)      # 금속층 (d3): 5 ~ 40 nm, 간격 5 nm → 총 8개

print(len(d1_list_nm)*len(d2_list_nm)*len(d3_list_nm))  # 총 조합 개수 출력 (8 × 187 × 8 = 11,968)

# ────────────────────────────────────────────────────────────────────────────
# (5) 샘플 저장용 리스트 초기화
# ────────────────────────────────────────────────────────────────────────────

all_d_tilde = []    # ▶ 각 샘플의 (d1, d2, d3) 두께 조합을 저장할 리스트
                    #   일반적으로 나중에 정규화된 형태로 저장되며 학습 입력으로 사용

all_T_target = []   # ▶ 각 조합에 대응하는 스펙트럼 T(λ) [shape: (401,)] 를 저장할 리스트
                    #   Forward model을 통해 계산된 정답 스펙트럼 (target)


8384


In [ ]:
# ────────────────────────────────────────────────────────────────────────────
# (A-1) TMMNetwork 정의 (클래스 이름, dtype 통일, calculate() 추가)
# ────────────────────────────────────────────────────────────────────────────
class TMMNetwork(nn.Module):
    def __init__(self, n_list_input, n_i_input, n_s_input, wavelengths_m_tensor):
        super().__init__()
        dtype_complex = torch.complex128
        dtype_float = torch.float64

        n_list_real = n_list_input.real
        n_list_imag = -torch.abs(n_list_input.imag)
        self.register_buffer('n_list', torch.complex(n_list_real, n_list_imag))
        self.register_buffer('n_i', n_i_input.to(dtype_complex))
        self.register_buffer('n_s', n_s_input.to(dtype_complex))

        # 파수 k0
        k0 = 2 * torch.pi / torch.clamp(wavelengths_m_tensor, min=1e-20)
        self.register_buffer('k0', k0)  # 이제 k0_tilde 아님

        self.num_layers = self.n_list.shape[0]
        self.num_wavelengths = wavelengths_m_tensor.shape[0]

    def forward(self, thicknesses_nm):
        device = thicknesses_nm.device
        dtype_complex = torch.complex128
        imag_unit = torch.tensor(1j, dtype=dtype_complex, device=device)

        if thicknesses_nm.ndim == 1:
            thicknesses_nm = thicknesses_nm.unsqueeze(0)  # (1, 3)
        B = thicknesses_nm.shape[0]  # batch size

        # Convert to meters
        thicknesses_m = thicknesses_nm * 1e-9  # (B, 3)

        # (B, 3, 401): broadcasting layer × wavelength
        delta = thicknesses_m[:, :, None] * self.k0[None, None, :] * self.n_list[None, :, :]

        cosδ = torch.cos(delta)
        sinδ = torch.sin(delta)

        # 초기 M_total: (B, 401, 2, 2)
        M_total = torch.eye(2, dtype=dtype_complex, device=device).repeat(B, self.num_wavelengths, 1, 1)

        for j in range(self.num_layers):
            Yj = self.n_list[j] * 2.654e-3  # (401,)

            sin_j = sinδ[:, j, :]
            cos_j = cosδ[:, j, :]

            m11 = cos_j
            m12 = imag_unit * sin_j / Yj[None, :]
            m21 = imag_unit * Yj[None, :] * sin_j
            m22 = cos_j

            Mj = torch.stack([
                torch.stack([m11, m12], dim=-1),
                torch.stack([m21, m22], dim=-1)
            ], dim=-2)  # shape: (B, 401, 2, 2)

            M_total = torch.matmul(M_total, Mj)

        Y_in = self.n_i * 2.654e-3
        Y_out = self.n_s * 2.654e-3

        B_ = M_total[:, :, 0, 0] + M_total[:, :, 0, 1] * Y_out
        C_ = M_total[:, :, 1, 0] + M_total[:, :, 1, 1] * Y_out
        denom = Y_in * B_ + C_
        t1 = (2 * Y_in) / denom  # (B, 401)

        T = (torch.real(self.n_s) / torch.real(self.n_i)) * torch.abs(t1) ** 2
        return T.to(torch.float64)  # (B, 401)

model = TMMNetwork(n_list_torch, n_i_torch, n_s_torch, λ_tensor_global).to(device)

In [ ]:
import os
import random
import numpy as np
import torch

# ------------------------------
# 시드 고정 (재현성 확보)
# ------------------------------
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)  # 시드 고정 실행

In [ ]:
# ──────────────────────────────────────────────
# (A) 데이터 생성: (두께 → 스펙트럼) via TMM
# ──────────────────────────────────────────────
from torch.utils.data import Dataset, DataLoader, random_split

all_d = []           # ▶ 각 샘플의 [d1, d2, d3] 두께 조합 (정규화 전, nm)
all_T_target = []    # ▶ 각 샘플에 대한 T(λ) 스펙트럼 (shape: [401])

print("Generating (normalized_d → T_target) pairs ...")

for d1 in d1_list_nm:
    for d2 in d2_list_nm:
        for d3 in d3_list_nm:
            d_nm_vec = torch.tensor([[d1, d2, d3]], dtype=torch.float64)  # shape: [1, 3]
            all_d.append(d_nm_vec.cpu().numpy().squeeze(0))  # shape: [3]

            T_spec = model(d_nm_vec.to(device))              # TMM forward (입력: nm 단위 두께)
            all_T_target.append(T_spec.detach().cpu().numpy())

# ▶ NumPy 배열로 변환 (shape: [N, 3] / [N, 401])
all_d = np.array(all_d, dtype=np.float64)
all_T_target = np.array(all_T_target, dtype=np.float64)
print("Total samples:", all_d.shape[0])

# ──────────────────────────────────────────────
# (B) 두께 정규화 (Min-Max Scaling)
# ──────────────────────────────────────────────
THICKNESS_MIN = torch.tensor(np.min(all_d, axis=0), dtype=torch.float64, device=device)
THICKNESS_MAX = torch.tensor(np.max(all_d, axis=0), dtype=torch.float64, device=device)

def standardize_thickness(d_nm_array):
    """입력: 실제 두께 (nm), 출력: 정규화된 두께 [0,1]"""
    d_nm_array = torch.tensor(d_nm_array, dtype=torch.float64, device=device) if not isinstance(d_nm_array, torch.Tensor) else d_nm_array.to(dtype=torch.float64, device=device)
    return (d_nm_array - THICKNESS_MIN) / (THICKNESS_MAX - THICKNESS_MIN)

def destandardize_thickness(d_norm_array):
    """입력: 정규화된 두께, 출력: 실제 두께 (nm)"""
    d_norm_array = d_norm_array.clone().to(dtype=torch.float64, device=device)
    return d_norm_array * (THICKNESS_MAX - THICKNESS_MIN) + THICKNESS_MIN

all_d_norm = standardize_thickness(all_d)                     # shape: [N, 3]
d_nm_check = destandardize_thickness(all_d_norm)              # 역변환 확인용

# ──────────────────────────────────────────────
# (C) 스펙트럼 정규화 (Min-Max Scaling per λ)
# ──────────────────────────────────────────────
SPECTRUM_MIN = torch.tensor(np.min(all_T_target, axis=0), dtype=torch.float64).to(device)
SPECTRUM_MAX = torch.tensor(np.max(all_T_target, axis=0), dtype=torch.float64).to(device)

def standardize_spectrum(T_array):
    """입력: 원본 스펙트럼, 출력: [0,1] 정규화"""
    T_tensor = torch.tensor(T_array, dtype=torch.float64).to(device)
    return (T_tensor - SPECTRUM_MIN) / (SPECTRUM_MAX - SPECTRUM_MIN)

def destandardize_spectrum(T_std_array):
    """입력: 정규화된 스펙트럼, 출력: 원본 스펙트럼"""
    return T_std_array * (SPECTRUM_MAX - SPECTRUM_MIN) + SPECTRUM_MIN

# ▶ 정규화 적용 + 역변환 체크
all_T_target_std = standardize_spectrum(all_T_target)
restored_T = destandardize_spectrum(all_T_target_std)
assert np.allclose(restored_T.cpu().numpy(), all_T_target, atol=1e-5)

# ──────────────────────────────────────────────
# (D) PyTorch Dataset 정의
# ──────────────────────────────────────────────
class BiTMMNormalizedDataset(Dataset):
    """
    Bi-directional TMM용 데이터셋 클래스.
    입력: 정규화된 두께
    출력: 정규화된 스펙트럼
    """
    def __init__(self, d_norm_array, T_array):
        self.d_norm = d_norm_array.clone().to(dtype=torch.float64)
        self.T_spec = T_array.clone().detach().to(dtype=torch.float64) if isinstance(T_array, torch.Tensor) else torch.tensor(T_array, dtype=torch.float64)

    def __len__(self):
        return self.d_norm.shape[0]

    def __getitem__(self, idx):
        return {
            'd_norm': self.d_norm[idx],       # shape: [3]
            'T_target': self.T_spec[idx]      # shape: [401]
        }

# ──────────────────────────────────────────────
# (E) Dataset 분할 및 DataLoader 구성
# ──────────────────────────────────────────────
batch_size = 32
seed = 42
g = torch.Generator().manual_seed(seed)  # 시드 고정

# ▶ 전체 Dataset 생성
dataset = BiTMMNormalizedDataset(all_d_norm, all_T_target_std)

# ▶ 비율 기준으로 Split (train/val/test = 90/5/5)
total_size = len(dataset)
train_size = int(0.9 * total_size)
val_size   = int(0.05 * total_size)
test_size  = total_size - train_size - val_size

print(f"Total: {total_size}, Train: {train_size}, Val: {val_size}, Test: {test_size}")

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size], generator=g
)

# ▶ DataLoader 정의 (GPU 훈련 대비 `pin_memory=True` 권장)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  num_workers=0, pin_memory=True, generator=g)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, pin_memory=True)


Generating (normalized_d → T_target) pairs ...
Total samples: 8384
Total: 8384, Train: 7545, Val: 419, Test: 420


In [ ]:
# ──────────────────────────────────────────────
# (추가) 전체 학습 데이터를 Pandas로 출력하기
# ──────────────────────────────────────────────

# (1) 역정규화 thickness → 실제 nm로 변환
d_real = destandardize_thickness(dataset.d_norm).cpu().numpy().squeeze()  # shape: (N, 3)

# (2) spectrum은 아직 standardized 상태이므로 복원
T_real = destandardize_spectrum(dataset.T_spec).cpu().numpy().squeeze()  # shape: (N, 401)

# (3) Pandas DataFrame으로 결합
df_thickness = pd.DataFrame(d_real, columns=['d1_nm', 'd2_nm', 'd3_nm'])
df_spectrum = pd.DataFrame(T_real, columns=[f"T_{w}nm" for w in WAVELENGTHS])

# (4) 결합
df_full = pd.concat([df_thickness, df_spectrum], axis=1)

# (5) 상위 5개 샘플 출력
print(df_full.head())

# (6) 전체 저장하고 싶다면 (선택)
df_full.to_csv("full_training_data.csv", index=False)

   d1_nm  d2_nm  d3_nm   T_380nm   T_381nm   T_382nm   T_383nm   T_384nm  \
0    5.0  150.0    5.0  0.664733  0.664586  0.664638  0.664841  0.665255   
1    5.0  150.0   10.0  0.600258  0.601023  0.602018  0.603169  0.604548   
2    5.0  150.0   15.0  0.521428  0.522529  0.523897  0.525447  0.527224   
3    5.0  150.0   20.0  0.437933  0.438960  0.440282  0.441814  0.443557   
4    5.0  150.0   25.0  0.357522  0.358260  0.359306  0.360578  0.362035   

    T_385nm   T_386nm  ...   T_771nm   T_772nm   T_773nm   T_774nm   T_775nm  \
0  0.665836  0.666619  ...  0.531328  0.532044  0.532772  0.533509  0.534252   
1  0.606091  0.607852  ...  0.410478  0.411625  0.412787  0.413959  0.415137   
2  0.529156  0.531305  ...  0.292363  0.293426  0.294502  0.295586  0.296675   
3  0.445438  0.447520  ...  0.199030  0.199842  0.200662  0.201487  0.202316   
4  0.363610  0.365360  ...  0.132318  0.132886  0.133460  0.134037  0.134616   

    T_776nm   T_777nm   T_778nm   T_779nm   T_780nm  
0  0.535

In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader
import os

# ░░ 0. Seed 고정 함수 ░░
def seed_everything(seed: int = 42):
    """
    모든 난수 생성기(random, numpy, torch)에 시드를 고정해
    실험 재현성을 보장함. 특히 GPU 연산에서도 순서를 고정하기 위해
    cudnn 옵션도 조정함.
    """
    import random, numpy as np
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# ░░ 1. 고정 하이퍼파라미터 설정 ░░
SEED = 42
LR = 6.639623079859457e-05         # 학습률: 옵튜나가 찾은 최적 값
ALPHA = 0.13940346079873228        # thickness loss의 가중치 (joint loss)
N_LAYERS = 4                       # MLP 은닉층 개수
N_UNITS = 768                      # 각 은닉층의 뉴런 수
BATCH_SIZE = 32
BETA = 14                          # thickness loss를 joint로 포함할 에폭 수
NUM_EPOCHS = 600                   # 총 학습 epoch 수
SAVE_DIR = "./checkpoints"        # 모델 저장 폴더
SAVE_PATH = os.path.join(SAVE_DIR, "inverse_best.pth")  # 저장 경로

# 디렉토리 생성 및 시드 고정
os.makedirs(SAVE_DIR, exist_ok=True)
seed_everything(SEED)
g = torch.Generator().manual_seed(SEED)

# ░░ 2. DataLoader 구성 ░░
# 사전 정의된 dataset / split 비율 사용
train_dataset, val_dataset, _ = random_split(
    dataset, [train_size, val_size, test_size], generator=g
)

# 학습/검증용 DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,  num_workers=0, generator=g)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# ░░ 3. InverseNet 정의 ░░
class InverseNet(nn.Module):
    """
    입력: 401 포인트의 스펙트럼 벡터
    출력: 정규화된 두께 벡터 [d1, d2, d3] ∈ [0, 1]
    구조: [Linear → LeakyReLU] × N → Linear → Sigmoid
    """
    def __init__(self, input_dim: int, output_dim: int, num_hidden_layers: int, units: int):
        super().__init__()
        layers = [nn.Linear(input_dim, units), nn.LeakyReLU()]  # 첫 층
        for _ in range(num_hidden_layers):
            layers += [nn.Linear(units, units), nn.LeakyReLU()]  # 반복 은닉층
        layers += [nn.Linear(units, output_dim), nn.Sigmoid()]   # 출력층
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# 모델 생성 및 GPU 할당
NUM_LAYERS_OUTPUT = len(material_sequence)  # 예: 3층 구조면 3
inverse_net = InverseNet(
    input_dim=NUM_WAVELENGTHS, output_dim=NUM_LAYERS_OUTPUT,
    num_hidden_layers=N_LAYERS, units=N_UNITS
).to(device).to(torch.float64)

# 옵티마이저 정의
optimizer = optim.Adam(inverse_net.parameters(), lr=LR)

# ░░ 4. 학습 루프 시작 ░░
print("\n[INFO] Start training InverseNet (best params)\n")
best_val_loss = float('inf')  # 현재까지 가장 낮은 검증 손실 저장

for epoch in range(1, NUM_EPOCHS + 1):

    # (1) ░ 학습 단계 ░
    inverse_net.train()
    epoch_loss = 0.0

    for batch in train_loader:
        T_target = batch['T_target'].to(device)        # 정규화된 스펙트럼 입력
        d_norm_true = batch['d_norm'].to(device)       # 정규화된 정답 두께

        optimizer.zero_grad()
        x = T_target.squeeze(1)                        # (B, 1, 401) → (B, 401)
        d_norm_pred = inverse_net(x)                  # 정규화된 두께 예측 (B, 3)

        # 예측된 두께 → 비정규화(nm 단위)
        d_nm_pred = destandardize_thickness(d_norm_pred)  # (B, 3)

        # forward TMM 모델로 스펙트럼 예측
        T_pred_batch = model(d_nm_pred)               # (B, 401)

        # 입력 스펙트럼 (정규화된 x)도 역정규화
        T_target_destd = destandardize_spectrum(x)    # (B, 401)

        # (a) spectrum loss (역정규화된 T 기준)
        loss_spectrum = F.mse_loss(T_pred_batch, T_target_destd)

        # (b) thickness loss (정규화 상태에서 직접 비교)
        loss_thickness = F.mse_loss(d_norm_pred, d_norm_true.squeeze(1))

        # (c) epoch에 따라 joint loss vs spectrum only
        if epoch <= BETA:
            loss = loss_spectrum + ALPHA * loss_thickness
        else:
            loss = loss_spectrum

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # (2) ░ 검증 단계 ░
    inverse_net.eval()
    val_loss_total = 0.0
    with torch.no_grad():
        for val_batch in val_loader:
            T_val = val_batch['T_target'].to(device)
            d_norm_val = val_batch['d_norm'].to(device)

            x_val = T_val.squeeze(1).to(torch.float64)
            d_pred_val = inverse_net(x_val)
            d_nm_val_pred = destandardize_thickness(d_pred_val)
            T_val_pred = model(d_nm_val_pred)
            T_val_destd = destandardize_spectrum(x_val)

            loss_spectrum_v = F.mse_loss(T_val_pred, T_val_destd)
            loss_thickness_v = F.mse_loss(d_pred_val, d_norm_val.squeeze(1))
            val_loss = (loss_spectrum_v + ALPHA * loss_thickness_v) if epoch <= BETA else loss_spectrum_v
            val_loss_total += val_loss.item()

    # (3) ░ 로그 출력 + Best 모델 저장 ░
    avg_train_loss = epoch_loss / len(train_loader)
    avg_val_loss = val_loss_total / len(val_loader)

    print(f"Epoch[{epoch}/{NUM_EPOCHS}]  train: {avg_train_loss:.6f}  val: {avg_val_loss:.6f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(inverse_net.state_dict(), SAVE_PATH)  # 가장 좋은 모델 저장



[INFO] Start training InverseNet (best params)



KeyboardInterrupt: 

# **학습 시킨 모델 불러오기**

In [ ]:
import torch
import torch.nn as nn
import urllib.request
import os

# ░░ 모델 정의 ░░
class InverseNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 768), nn.LeakyReLU(),
            nn.Linear(768, 768), nn.LeakyReLU(),
            nn.Linear(768, 768), nn.LeakyReLU(),
            nn.Linear(768, 768), nn.LeakyReLU(),
            nn.Linear(768, 768), nn.LeakyReLU(),
            nn.Linear(768, output_dim), nn.Sigmoid()
        )
    def forward(self, x):
        return self.net(x)

# ░░ 모델 인스턴스 ░░
inverse_net = InverseNet(input_dim=NUM_WAVELENGTHS, output_dim=len(material_sequence)).to(device).to(torch.float64)

# ░░ GitHub에서 모델 다운로드 ░░
github_url = "https://raw.githubusercontent.com/KIMEUIJOON-KNU/Bi-directional/main/inverse_best_New.pth"
local_model_path = "inverse_best_New.pth"

try:
    urllib.request.urlretrieve(github_url, local_model_path)
    print("✅ 모델 파일 다운로드 성공")
except Exception as e:
    print("❌ 다운로드 실패:", e)

# ░░ 모델 파라미터 로드 ░░
try:
    inverse_net.load_state_dict(torch.load(local_model_path, map_location=device))
    print("✅ 모델 파라미터 로드 성공")
except Exception as e:
    print("❌ 모델 로드 실패:", e)


✅ 모델 파일 다운로드 성공
✅ 모델 파라미터 로드 성공


In [45]:
import torch
import torch.nn as nn
import urllib.request
import os

# ░░ 모델 정의 ░░
class InverseNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 768), nn.LeakyReLU(),
            nn.Linear(768, 768), nn.LeakyReLU(),
            nn.Linear(768, 768), nn.LeakyReLU(),
            nn.Linear(768, 768), nn.LeakyReLU(),
            nn.Linear(768, 768), nn.LeakyReLU(),
            nn.Linear(768, output_dim), nn.Sigmoid()
        )
    def forward(self, x):
        return self.net(x)

# ░░ 모델 인스턴스 ░░
inverse_net = InverseNet(input_dim=NUM_WAVELENGTHS, output_dim=len(material_sequence)).to(device).to(torch.float64)

# ░░ GitHub에서 모델 다운로드 ░░
github_url = "https://raw.githubusercontent.com/KIMEUIJOON-KNU/Bi-directional/main/inverse_best_New.pth"
local_model_path = "inverse_best_New.pth"

try:
    urllib.request.urlretrieve(github_url, local_model_path)
    print("✅ 모델 파일 다운로드 성공")
except Exception as e:
    print("❌ 다운로드 실패:", e)

# ░░ 모델 파라미터 로드 ░░
try:
    inverse_net.load_state_dict(torch.load(local_model_path, map_location=device))
    print("✅ 모델 파라미터 로드 성공")
except Exception as e:
    print("❌ 모델 로드 실패:", e)

# ░░ (4) test set만 추출 ░░
_, _, test_dataset = random_split(dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(42))

# ░░ (5) 예측 실행 (전체 test set) ░░
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

all_true_d = []
all_pred_d = []

with torch.no_grad():
    for batch in test_loader:
        T_test = batch['T_target'].to(device).squeeze(1)
        d_true_norm = batch['d_norm'].to(device)

        # 예측 (정규화된 두께)
        d_pred_norm = inverse_net(T_test)

        # 정규화된 두께 → 실제 두께(nm)
        d_pred_nm = destandardize_thickness(d_pred_norm).cpu().numpy()
        d_true_nm = destandardize_thickness(d_true_norm).cpu().numpy()

        all_true_d.append(d_true_nm)
        all_pred_d.append(d_pred_nm)

# ░░ (6) 결과 병합 및 DataFrame 출력 ░░
true_all = torch.tensor(np.concatenate(all_true_d, axis=0))
pred_all = torch.tensor(np.concatenate(all_pred_d, axis=0))

df_result = pd.DataFrame({
    "True_d1": true_all[:, 0],
    "True_d2": true_all[:, 1],
    "True_d3": true_all[:, 2],
    "Pred_d1": pred_all[:, 0],
    "Pred_d2": pred_all[:, 1],
    "Pred_d3": pred_all[:, 2],
})

print(df_result.head(10))  # 상위 10개 결과 출력
excel_path = "inverse_test_result.xlsx"
df_result.to_excel(excel_path, index=False)
print(f"저장 완료: {excel_path}")

✅ 모델 파일 다운로드 성공
✅ 모델 파라미터 로드 성공
   True_d1  True_d2  True_d3    Pred_d1     Pred_d2    Pred_d3
0     15.0    430.0     20.0  14.825414  429.895890  20.286923
1     25.0    685.0     30.0  25.737539  684.942827  29.671310
2     10.0    485.0      5.0   8.254471  486.948506   6.991958
3     15.0    630.0     40.0  15.063486  630.098762  39.918744
4     15.0    385.0     25.0  14.593657  384.706214  25.016640
5     10.0    440.0     20.0   9.734059  439.415833  20.101497
6     40.0    615.0     35.0  36.105777  615.054982  37.982515
7     30.0    285.0      5.0   7.685507  294.291328  26.966713
8     25.0    220.0     30.0  25.216201  219.926069  29.712750
9     30.0    420.0     30.0  29.973181  419.944812  29.953681
저장 완료: inverse_test_result.xlsx
